In [1]:
from enum import IntEnum
from typing import List, Union, Tuple

class Lollipop(IntEnum):
    RED = 0
    YELLOW = 1

In [2]:
state_map = (
    # input: Red=0, Yellow=1
    # 0, 1
    ( 1, 5 ), # state 0: Start state
    ( 2, 5 ), # state 1: 1st RED
    ( 3, 5 ), # state 2: 2nd RED
    ( 4, 5 ), # state 3: 3rd RED
    ( 4, 5 ), # state 4: RED>=4
    ( 1, 6 ), # state 5: 1st Yellow
    ( 1, 7 ), # state 6: 2nd Yellow
    ( 1, 8 ), # state 7: 3rd Yellow
    ( 1, 8 ), # state 8: Yellow>=4
)


In [3]:

def iterate_machine(
        input: List[int], state_map: Tuple[Tuple[int]], 
        start_state=0):
    """
    A generator, that iterates the states of the machine  
    """
    state = start_state
    index = 0
    len_input = len(input)

    while(index < len_input):
        eval = input[index]
        state = state_map[state][eval]

        yield state

        index+=1




In [4]:
def iterate_factory_line(input: List[int], start_state=0):
    for state in iterate_machine(input, state_map=state_map, start_state=start_state):
        if state==3:
            print('Oh No! Too many RED Lolipops :(')
        elif state==7:
            print('Oh No! Too many Yellow Lolipops :(')
    
    print('Run finished.')


In [5]:
# Line examples from the PDF, translated into arrays
LINE_1 = [0, 1, 0, 1, 0, 1, 0, 1, 0]
LINE_2 = [0, 1, 0, 0, 1, 0, 1, 0]
LINE_3 = [0, 1, 0, 0, 0]
LINE_4 = [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [6]:
def test(tests=[LINE_1, LINE_2, LINE_3, LINE_4]):
    for ix, line in enumerate(tests):
        print(f'- Line {ix + 1} is starting')
        print(line)
        iterate_factory_line(line)

In [7]:
test()

- Line 1 is starting
[0, 1, 0, 1, 0, 1, 0, 1, 0]
Run finished.
- Line 2 is starting
[0, 1, 0, 0, 1, 0, 1, 0]
Run finished.
- Line 3 is starting
[0, 1, 0, 0, 0]
Oh No! Too many RED Lolipops :(
Run finished.
- Line 4 is starting
[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1]
Oh No! Too many RED Lolipops :(
Oh No! Too many Yellow Lolipops :(
Run finished.


In [28]:
class State(object):
    _id_counter: int = 0

    def __new__(cls):
        print(f"Creating a new state object {State._id_counter}")
        instance = super().__new__(cls)
        State._id_counter += 1
        return instance
    
    def __init__(self, data: object=None):
        self.__id = State._id_counter
        self.__data = data

    def id(self):
        return self.__id

    def data(self):
        return self.__data


class Event():
    _id_counter: int = 0

    def __init__(self, id: int, data: object):
        self.__id = id
        self.__data = data

from typing import List

class Machine(object):

    def __init__(self, state=None):
        self._states_dict = {}
        self._state = state

    def add(self, from_state: object, event: object, to_state: object):
        events_dict = self._states_dict.get(from_state, {})
        events_dict.update({ event : to_state })
        self._states_dict.update({from_state : events_dict})
    
    def iterate_input(self, input: List[object]):
        index = 0
        len_input = len(input)

        while(index < len_input):
            eval = input[index]
            events_dict = self._states_dict.get(self._state, None)

            if events_dict is None:
                raise Exception(f'No state registered for {self._state}')
            
            sync_state = events_dict.get(eval, None)

            if sync_state is None:
                raise Exception(f'No sync state registered for event {eval}')

            self._state = sync_state

            yield self._state

            index+=1




In [29]:
state_map = (
    # input: Red=0, Yellow=1
    # 0, 1
    ( 1, 5 ), # state 0: Start state
    ( 2, 5 ), # state 1: 1st RED
    ( 3, 5 ), # state 2: 2nd RED
    ( 4, 5 ), # state 3: 3rd RED
    ( 4, 5 ), # state 4: RED>=4
    ( 1, 6 ), # state 5: 1st Yellow
    ( 1, 7 ), # state 6: 2nd Yellow
    ( 1, 8 ), # state 7: 3rd Yellow
    ( 1, 8 ), # state 8: Yellow>=4
)


In [38]:
m = Machine("Start")

m.add("Start", "red", "R1")
m.add("Start", "yellow", "Y1")

m.add("R1", "red", "R2")
m.add("R1", "yellow", "Y1")

m.add("R2", "red", "R3")
m.add("R2", "yellow", "Y1")

m.add("R3", "red", "R4")
m.add("R3", "yellow", "Y1")

m.add("R4", "red", "R4")
m.add("R4", "yellow", "Y1")

#

m.add("Y1", "red", "R1")
m.add("Y1", "yellow", "Y2")

m.add("Y2", "red", "R1")
m.add("Y2", "yellow", "Y3")

m.add("Y3", "red", "R1")
m.add("Y3", "yellow", "Y4")

m.add("Y4", "red", "R1")
m.add("Y4", "yellow", "Y4")

m

In [39]:
A = "red"
B = "yellow"

LINE_1 = [A, B, A, B, A, B, A, B, A]
LINE_2 = [A, B, A, A, B, A, B, A]
LINE_3 = [A, B, A, A, A]
LINE_4 = [A, B, A, A, A, A, A, B, B, B, B]

In [44]:

def iterate_factory_line2(input: List[int], start_state=0):
    for state in m.iterate_input(input):
        if state=="R3":
            print('Oh No! Too many RED Lolipops :(')
        elif state=="Y3":
            print('Oh No! Too many Yellow Lolipops :(')
    
    print('Run finished.')


In [49]:
iterate_factory_line2(LINE_4)

Oh No! Too many RED Lolipops :(
Oh No! Too many Yellow Lolipops :(
Run finished.
